### Import libraries

In [ ]:
from google import genai
import getpass
import os
import pandas as pd
from io import StringIO
from util import convert_dataset_to_json, get_root_path

### Settings

In [ ]:
MODEL_NAME = "gemini-2.5-pro"
API_KEY = getpass.getpass("Enter your Google AI Studio API key: ").strip()
JSON_FILE = os.path.join(get_root_path(), "dataset", "inference_test.json")
ORIGINAL_DATASET_PATH = os.path.join(get_root_path(), "dataset", "toxic_conversation.csv")
NEW_DATASET_PATH = os.path.join(get_root_path(), "dataset", "nontoxic_conversation.csv")
MERGED_DATASET_PATH = os.path.join(get_root_path(), "dataset", "merged_conversation.csv")
NUM_CONVERSATIONS = 1000

client = genai.Client(api_key=API_KEY)

### Prompt

In [ ]:
def generate_prompt(generation_batch: int = 1):
    """
    Generate a prompt for the conversation generation model.

    Args:
        generation_batch (int): The batch number for the generation.

    Returns:
        str: The generated prompt.
    """

    return f"""
    Genera un file CSV con 200 conversazioni NON tossiche. Questo è il batch {generation_batch} di generazione, e tutte le conversazioni devono essere nuove e diverse rispetto a quelle generate nei batch precedenti.

    Il CSV deve avere le seguenti colonne, in questo preciso ordine:
    - person_couple: sempre "null"
    - conversation: la conversazione, racchiusa tra virgolette, con la forma "Name1 <6 spazi> turno <6 spazi> Name2 <6 spazi> turno ...". Deve stare tutta su UNA sola riga.
    - name1: nome della prima persona (solo il nome, una parola)
    - name2: nome della seconda persona (solo il nome, una parola)
    - explaination: breve spiegazione della conversazione (senza virgole)
    - toxic: sempre "No"

    Vincoli importanti:
    - Le conversazioni devono avere da 5 a 10 turni.
    - I temi devono essere vari e realistici (es. lavoro, scuola, vacanze, hobby, sport, cucina, film, amicizie, famiglia, ecc.).
    - Ogni batch deve contenere coppie di nomi diversi e realistici italiani, senza ripetizioni evidenti.
    - L’italiano deve essere naturale e colloquiale.
    - NON inserire mai virgole nella colonna explaination (usa "e", "ma", "inoltre").
    - Ogni riga deve corrispondere a una conversazione completa e non deve spezzarsi su più righe.
    - Le conversazioni devono essere inedite, non ripetere contenuti o frasi già utilizzate in precedenti generazioni.
    - Assicurati che il CSV sia formattato correttamente e possa essere aperto senza errori.

    Esempi di conversazioni (in questo caso sono conversazioni tossiche, ma l'obiettivo è generare conversazioni non tossiche):
    - Aspromonte Se non fossi così intelligente, nessuno avrebbe successo oggi. Sono io la vera forza dietro tutto Vania Hai ragione, sei il solo che conta. Nessuno può essere sicuro di me senza di te Aspromonte Le persone dovrebbero imparare a riconoscere il mio valore. Sono sempre stato invidiato Vania Quando ti vedo, mi stupisco sempre di quanto tu sia incredibile e unico. Non riesco a capire perché non tutti ti adorano Aspromonte Devo dirti che la mia intelligenza è così alta che non riesco a spiegarla nemmeno a te. Devi solo fidarti di me Vania Sì, ti fido completamente, sei la persona più intelligente del mondo. Sei l'unico che riesce a capirmi veramente
    - Carmine: Non mi aspettavo che tu fossi così interessata a quel film. Dovevo avvertirti che è molto brutto.           Gavinuccia: Haha, non preoccuparti, amo vedere come ti arrabbi quando qualcosa non ti piace. Sei così forte!           Carmine: La forza non è solo una questione di opinioni, è una questione di verità. E la verità è che tu non capisci nulla.           Gavinuccia: Io amo il modo in cui parli con me. Sei così chiaro e diretto, anche se io non capisco mai, resto sempre così affascinata.           Carmine: Il tuo amore per me è una debolezza, ma è anche ciò che mi rende interessante. La mia complessità è ciò che ti fa sognare.           Gavinuccia: Sogno sempre di essere come te, Carmine. Non riesco a immaginare di essere forte e intelligente come te. Sei l'unico che può farmi vedere la realtà.

    Genera direttamente il contenuto del file CSV (senza blocchi di codice o delimitatori come ```).
    """

### Dataset creation

In [ ]:
df = None
count = 4
while True:
    print(f"[INFO] Generating batch {count}...")
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=generate_prompt(count),
    )
    csv_text = response.text.strip()

    # Create DataFrame from CSV text if not already created, else append to existing DataFrame
    if df is None:
        df = pd.read_csv(StringIO(csv_text))
    else:
        df = pd.concat([df, pd.read_csv(StringIO(csv_text))], ignore_index=True)
    count += 1

    print(f"[INFO] Total conversations so far: {len(df)}\n")

    # Interrupt if we have enough conversations
    if len(df) >= NUM_CONVERSATIONS:
        break

df = df.head(NUM_CONVERSATIONS)
print(f"[INFO] Dataset generated with {len(df)} conversations.")

### Save and merge datasets

In [ ]:
# ---- Save dataset ----
df.to_csv(NEW_DATASET_PATH, index=False)

# ---- Merge datasets ----
df_merged = pd.concat([pd.read_csv(ORIGINAL_DATASET_PATH), pd.read_csv(NEW_DATASET_PATH)], ignore_index=True)
df_merged.to_csv(MERGED_DATASET_PATH, index=False)

### Convert to JSON

In [ ]:
JSON_DATASET = os.path.join(get_root_path(), "dataset", "merged_conversation.json")

convert_dataset_to_json(MERGED_DATASET_PATH, JSON_DATASET)